In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Veri Setini okuma
data = pd.read_csv("/kaggle/input/groceries-dataset/Groceries_dataset.csv")

In [ ]:
# verinin ilk bakış
data.head()

In [ ]:
# Boyut bilgisi veri setimiz 38765 satır ve 3 sütundan oluşmaktadır
data.shape

In [ ]:
# veri seti hakkında bilgiler
data.info()

In [ ]:
# Veri setinde eksik gözlem olup olmadıgını kontrol etme -> Hiç eksik gözlem yokmuş
data.isnull().values.any()

In [ ]:
# Veri setine yeni sutun ekleme
data["Date"] = data["Date"].apply(pd.to_datetime)
data["day"] = data.Date.dt.dayofweek
data["day_name"] = data.Date.dt.day_name()
data["weekday"] = np.where(data["day"].isin([5,6]), 1,0)

yukarıdaki işlemde Date sutunun tipini datetime çevirdik ve daha sonra veri setinde iki tane sutun ekledik.Birinci sütun day sütunu ikincisi ise günün haftaiçi veya haftasonuna denk geldiğini bulan sutun

In [ ]:
data.head(10)

In [ ]:
# 728 benzersiz tarih var veri setimizde
len(data.Date.unique())

In [ ]:
# Veri setimizde 3898 tane üyemiz varmış
len(data["Member_number"].unique())

In [ ]:
# veri setinde toplamda 167 çeşit ürün var
len(data["itemDescription"].unique())

# Visualization

In [ ]:
# hangi gün toplam ne kadar iş olduğunu bulma
gunluk_satis = data.groupby(["day_name"])["itemDescription"].agg(["count"]).reset_index()
gunluk_satis.sort_values("day_name",ascending = True)

In [ ]:
plt.figure(figsize = (12,8))
sns.barplot(x ="day_name", y ="count",data = gunluk_satis,color = "Blue",label = "count")
plt.xlabel("Günler")
plt.ylabel("Adet")
plt.title("Günlere göre Satış")
plt.xticks(rotation = 60)
plt.legend()
plt.show()

In [ ]:
# Haftaiçi ve haftasonu satış miktarları
data.groupby(["weekday"])["itemDescription"].agg(["count"])

In [ ]:
# Haftaiçi ve Haftasonuna göre yapılan satışların toplamı
data.groupby(["weekday"])["itemDescription"].agg(["count"]).plot(figsize=(12,5),kind = "bar",color = "red")
plt.title("Haftaiçi ve Haftasonuna  Göre Satılan Toplam Ürün Sayısı")
plt.xlabel("Haftaiçi-Haftasonu")
plt.ylabel("Satılan Toplam Ürün Sayısı")
plt.show()

In [ ]:
# Tarihe Göre Satılan Toplam Ürün sayısı
tarih_satis = data.groupby(["Date"])["itemDescription"].agg(["count"]).reset_index()
tarih_satis

In [ ]:
plt.figure(figsize= (12,5))
sns.lineplot(x="Date", y="count", data=tarih_satis)
plt.xticks(rotation=60)
plt.grid()
plt.xlabel("Tarih")
plt.ylabel("Adet")
plt.title("Tarihe Göre Satışlar")
plt.show()

In [ ]:
tarih_satis = tarih_satis.sort_values(by="count",ascending=False)
tarih_satis[0:10]

In [ ]:
# Veri setimizde her bir ürünün frekansı 
data["itemDescription"].value_counts()

In [ ]:
# Ürünlerin Frekansları
plt.figure(figsize = (30,10))
plt.grid()
sns.barplot(x = data.itemDescription.value_counts().keys(),y = data.itemDescription.value_counts().values,color="Blue")
plt.title("Her Bir Ürünün Veri Setindeki Frekans Sayısı")
plt.xlabel("Ürünler")
plt.ylabel("Frekans")
plt.xticks(rotation=90)
plt.show()

# Apriori

In [ ]:
apriori_data = data.groupby(["Member_number","Date"])["itemDescription"].unique()
apriori_data

Not -> Yukarıdaki satırda bir üyenin hangi tarihte neler almış onları ayrı ayrı satırlarda tutmak yerine bir müşterinin aynı tarihte aldığı ürünleri toplam olarak 
tek bir satırda tuttuk.Bir müşteri aynı gün içerisinde iki veya daha fazla kez alışverişe gelmiş olabilir ama biz bunu göz ardı edip tek bir alışverişmiş gibi düşünüp
topluca tek satırda gösterdik.

In [ ]:
transactions_encoder = TransactionEncoder()
transactions_encoder_array = transactions_encoder.fit(apriori_data).transform(apriori_data)
transactions = pd.DataFrame(transactions_encoder_array, columns=transactions_encoder.columns_)

In [ ]:
transactions_encoder_array

In [ ]:
transactions_encoder.columns_

In [ ]:
transactions

In [ ]:
# Veri setindeki ürünlerin support değerlerinin hesaplanması support değeri 0.003 ten küçük olanları almaz
items = apriori(transactions, min_support=0.003, use_colnames=True, verbose=1)
a = items.sort_values("support",ascending = False)
a.head(10)

In [ ]:
rules = association_rules(items, metric="confidence", min_threshold=0.001)
rules.head()

In [ ]:
sns.heatmap(rules.corr(),annot = True,linewidths=2,linecolor = "red", fmt = '.1f')
plt.show()

In [ ]:
rules.sort_values('confidence', ascending = False, inplace=True)
rules.head(10)

In [ ]:
rules.sort_values('lift', ascending = False, inplace=True)
rules.head(10)

In [ ]:
rules.sort_values('support', ascending = False, inplace=True)
rules.head(10)